## Pandas (Vectorized String Operations)

Una de las fortalezas de Python es su relativa facilidad para manejar y manipular cadenas de texto. Pandas construye sobre ello una serie de operaciones vectorizadas que llegan a ser una pieza esencial del tipo de manipulación necesaria cuando uno está trabajando con datos reales.

In [65]:
import numpy as np
import pandas as pd

### Introducing Pandas String Operations

Numpy y Pandas tienen bien cubiertas las operaciones aritméticas, por lo que pueden realizar fácilmente operaciones a través de un array.

In [7]:
x = np.array([1,2,3,4,5,6])

In [8]:
x * 2

array([ 2,  4,  6,  8, 10, 12])

Estas operaciones vectorizadas simplifica la sintásis a la hora de operar a través de un array. Pero que pasa con las cadenas de caracteres?. Numpy no proporciona este tipo de facilidades y hay que realizar la operativa de un modo más manual

In [2]:
data = ['peter','Paul', 'MARY', 'gUIDO']

In [4]:
[s.capitalize() for s in data]

['Peter', 'Paul', 'Mary', 'Guido']

Esto tiene sus inconvenientes, por ejemplo, si dentro del array nos encontramos con missing values

In [9]:
data = ['peter','Paul', None, 'gUIDO']

In [10]:
[s.capitalize() for s in data]

AttributeError: 'NoneType' object has no attribute 'capitalize'

Pandas incorpora una serie de mejoras, que permite manejar los valores nulos, vía el astributo "str" de las Series de Pandas y los índices basados en strings.

In [15]:
# Creamos una serie con los datos anteriores
names = pd.Series(data)

In [16]:
names

0    peter
1     Paul
2     None
3    gUIDO
dtype: object

In [17]:
# podemos usar el método capitalize y Pandas ignorará aquellos valores
# nulos, no realizando la operación sobre ellos
names.str.capitalize()

0    Peter
1     Paul
2     None
3    Guido
dtype: object

### Tables of Pandas String Methods

Hay multitud de métodos a aplicar sobre cadenas de caracteres

#### Methods similar to Python string methods

len() upper() lower() isalfnum(), isupper(), split()
strip(), etc, etc

In [34]:
names.str.lower()

0    peter
1     paul
2     None
3    guido
dtype: object

In [21]:
names.str.isalpha()

0    True
1    True
2    None
3    True
dtype: object

In [22]:
names.str.split()

0    [peter]
1     [Paul]
2       None
3    [gUIDO]
dtype: object

In [51]:
names.str.startswith('p')

0     True
1    False
2     None
3    False
dtype: object

#### Methods using regular expressions

match() extract() findall() replace(), etc, etc.

In [52]:
data2 = ['peter parker parker','Paul parker', None, 'gUIDO parker']

In [53]:
y = pd.Series(data2)

In [54]:
y

0    peter parker parker
1            Paul parker
2                   None
3           gUIDO parker
dtype: object

In [55]:
y.str.findall('parker') # busca una palabra

0    [parker, parker]
1            [parker]
2                None
3            [parker]
dtype: object

In [56]:
y.str.count('parker') # cuenta las veces que aparece en el texto

0    2.0
1    1.0
2    NaN
3    1.0
dtype: float64

#### Methods using regular expressions

get(), slice(), repeat(), etc.

In [57]:
names.str.repeat(2) # repite el valor n veces

0    peterpeter
1      PaulPaul
2          None
3    gUIDOgUIDO
dtype: object

In [58]:
names.str.slice(0,3) # slicing de unas posiciones

0     pet
1     Pau
2    None
3     gUI
dtype: object

### Example Recipe Database

In [90]:
!curl -O https://s3.amazonaws.com/openrecipes/20170107-061401-recipeitems.json.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29.3M  100 29.3M    0     0  2427k      0  0:00:12  0:00:12 --:--:-- 2558k7 8161k    0     0  2198k      0  0:00:13  0:00:03  0:00:10 2197k 0  0:00:12  0:00:09  0:00:03 2321k


In [92]:
!gunzip 20170107-061401-recipeitems.json.gz

In [93]:
try:
    recipes = pd.read_json('20170107-061401-recipeitems.json')
except ValueError as e:
    print("ValueError:", e)

ValueError: Trailing data


In [97]:
with open('20170107-061401-recipeitems.json') as f:
    line = f.readline()


In [98]:
pd.read_json(line).shape

(2, 12)

In [109]:
with open('20170107-061401-recipeitems.json', 'r') as f:
    data = (line.strip() for line in f)
    data_json = "[{0}]".format(','.join(data))

recipes = pd.read_json(data_json)

In [110]:
recipes.shape

(173278, 17)

In [112]:
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
cookTime                                                          PT30M
creator                                                             NaN
dateModified                                                        NaN
datePublished                                                2013-03-11
description           Late Saturday afternoon, after Marlboro Man ha...
image                 http://static.thepioneerwoman.com/cooking/file...
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
name                                    Drop Biscuits and Sausage Gravy
prepTime                                                          PT10M
recipeCategory                                                      NaN
recipeInstructions                                                  NaN
recipeYield                                                          12
source                                                  thepione

In [113]:
recipes['ingredients'].str.len().describe()

count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

In [132]:
recipes['ingredients'].str.len().values.argmax()

135598

In [133]:
recipes['name'].iloc[recipes['ingredients'].str.len().values.argmax()]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

In [135]:
recipes['ingredients'].str.contains('[Cc]inamon').head()

0    False
1    False
2    False
3    False
4    False
Name: ingredients, dtype: bool

In [137]:
recipes['ingredients'].str.contains('[Cc]inamon').sum()

11

In [149]:
import re

In [138]:
spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley',
             'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']

In [161]:
recipes['ingredients'].str.contains('salt', re.IGNORECASE).head()

0    False
1    False
2     True
3    False
4    False
Name: ingredients, dtype: bool

In [159]:
x = ('salt', recipes['ingredients'].str.contains('salt', re.IGNORECASE))

In [160]:
type(x)

tuple

In [151]:
# creamos un diccionario a partir de la tupla
a = dict((spice, recipes['ingredients'].str.contains(spice, re.IGNORECASE)) for spice in spice_list)

In [162]:
# creamos el df a través del diccionario
spice_df = pd.DataFrame(a)

In [163]:
spice_df.head()

,salt,pepper,oregano,sage,parsley,rosemary,tarragon,thyme,paprika,cumin
0,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


In [169]:
# creamos un filtro de aquellas recetas que contienen 3 ingredientes
selection = spice_df.query('parsley & paprika & tarragon')

In [165]:
len(selection)

10

In [166]:
selection

,salt,pepper,oregano,sage,parsley,rosemary,tarragon,thyme,paprika,cumin
2069,False,True,False,False,True,False,True,False,True,False
74964,False,False,False,False,True,False,True,False,True,False
93768,True,True,False,True,True,False,True,False,True,False
113926,True,True,False,False,True,False,True,False,True,False
137686,True,True,False,False,True,False,True,False,True,False
140530,True,True,False,False,True,False,True,True,True,False
158475,True,True,False,False,True,False,True,False,True,True
158486,True,True,False,False,True,False,True,False,True,False
163175,True,True,True,False,True,False,True,False,True,False
165243,True,True,False,False,True,False,True,False,True,False


In [167]:
selection.index

Int64Index([2069, 74964, 93768, 113926, 137686, 140530, 158475, 158486, 163175,
            165243],
           dtype='int64')

In [170]:
# recuperamos el nombre de las recetas
recipes['name'][selection.index]

2069      All cremat with a Little Gem, dandelion and wa...
74964                         Lobster with Thermidor butter
93768      Burton's Southern Fried Chicken with White Gravy
113926                     Mijo's Slow Cooker Shredded Beef
137686                     Asparagus Soup with Poached Eggs
140530                                 Fried Oyster Po’boys
158475                Lamb shank tagine with herb tabbouleh
158486                 Southern fried chicken in buttermilk
163175            Fried Chicken Sliders with Pickles + Slaw
165243                        Bar Tartine Cauliflower Salad
Name: name, dtype: object